## 형태소 추출기 제작하기

### 사용할 형태소 분석기 불러오기

In [31]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum

# mac
# from konlpy.tag import Mecab

# 윈도우
import MeCab

# 트위터 형태소 분석기와 이름이 같음
from konlpy.tag import Okt


# 형태소 분석을 위한 객체 생성
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()
# 윈도우에서 Mecab 생성
mecab = MeCab.Tagger()

# 맥에서 Mecab 생성
# mecab = MeCab()

okt = Okt()

In [32]:
import re

def mecab_pos(text):
    pos = []
    
    # 우리가 원하는 TOKEN\tPOS 의 형태를 추출하는 정규 표현식
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정
    pos = [tuple(pattern.match(token).group(0).split("\t")) 
                   for token in mecab.parse(text).splitlines()[:-1]]
    
    return pos

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS 의 형태를 추출하는 정규 표현식
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정
    temp = [tuple(pattern.match(token).group(0).split("\t")) 
                   for token in mecab.parse(text).splitlines()[:-1]]
    
    # 추출한 토큰 중에 문자열만 선택
    for token in temp:
        morphs.append(token[0])  
    
    
    return morphs

def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS 의 형태를 추출하는 정규 표현식
    pattern = re.compile(".*\t[A-Z]+")
    
    
    temp = [tuple(pattern.match(token).group(0).split("\t")) 
                   for token in mecab.parse(text).splitlines()[:-1]]
    
    for token in temp:
        if token[1] == "NNG" or token[1]=="NNP" or token[1]=="NNB" or token[1]=="NNBC" or token[1]=="NP" or token[1]=="NR":
            nouns.append(token[0]) 
    return nouns

In [38]:
def text_cleaning(doc):
    
    # 한국어를 제외한 글자를 제거하는 함수.
    doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]","",doc)
    
    
    return doc


def define_stopwords(path):
    
    SW = set()
    
    # 불용어를 추가하는 방법 1.
    # SW.add("있다")    
    
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path,encoding='utf-8') as f:
        for word in f:
            SW.add(word)    
    
    return SW

def text_tokenizing(doc):
    
    
#     tokenized_doc = []
    
    # list comprehension을 풀어서 쓴 코드
#     for word in mecab_morphs(doc):
#         if word not in SW and len(word) > 1:
#             tokenized_doc.append(word)
#     return tokenized_doc
    
    
    return [word for word in mecab_morphs(doc) if word not in SW and len(word) > 1]

In [34]:
text3 = '''
2일 서울역으로 진입하던 무궁화호 열차 1량이 탈선하는 사고가 발생했다. 이날 오후 9시 25분 용산역을 출발하기 위해 회송 중이던 열차여서 승객은 타고 있지 않았다. 코레일 측은 "이번 사고로 인명피해는 없으나 경의선 전동열차의 운행구간을 일부 조정했다"고 설명했다.
한국철도공사(코레일)에 따르면 이날 오후 8시58분께 용산에서 여수엑스포역으로 출발하기 위해 회송하던 무궁화호 열차가 서울역 진입 중 맨 뒤에 달린 발전차 1량이 궤도를 이탈하는 사고가 났다. 이에 따라 이날 KTX와 일반열차는 정상운행 중이나 일부 경의선 열차 운행조정으로 지연이 발생하고 있는 것으로 알려졌다. 코레일은 사고 발생 즉시 긴급 복구반을 현장에 투입해 복구작업을 펼치고 있으며, 사고원인은 복구 이후 조사할 예정이다.
코레일 측은 "열차이용에 불편을 드린 점 깊이 사과드린다"며 "안전한 열차 운행과 복구를 위해 최선을 다하겠다"고 밝혔다.
'''

In [39]:
#

SW = define_stopwords("./stopwords-ko.txt")

cleaned_text = text_cleaning(text3)
print("전처리 : ",cleaned_text)

전처리 :  일서울역으로진입하던무궁화호열차량이탈선하는사고가발생했다이날오후시분용산역을출발하기위해회송중이던열차여서승객은타고있지않았다코레일측은이번사고로인명피해는없으나경의선전동열차의운행구간을일부조정했다고설명했다한국철도공사코레일에따르면이날오후시분께용산에서여수엑스포역으로출발하기위해회송하던무궁화호열차가서울역진입중맨뒤에달린발전차량이궤도를이탈하는사고가났다이에따라이날와일반열차는정상운행중이나일부경의선열차운행조정으로지연이발생하고있는것으로알려졌다코레일은사고발생즉시긴급복구반을현장에투입해복구작업을펼치고있으며사고원인은복구이후조사할예정이다코레일측은열차이용에불편을드린점깊이사과드린다며안전한열차운행과복구를위해최선을다하겠다고밝혔다


In [40]:
tokenized_text = text_tokenizing(cleaned_text)
print("\n형태소 분석 : ",tokenized_text)


형태소 분석 :  ['서울', '으로', '진입', '무궁화호', '차량', '탈선', '사고', '발생', '이날', '오후', '시분', '산역', '출발', '위해', '회송', '열차', '여서', '승객', '코레', '이번', '사고', '인명', '피해', '으나', '경의선', '전동', '열차', '운행', '구간', '일부', '조정', '다고', '설명', '한국', '철도', '공사', '레일', '따르', '이날', '오후', '시분', '용산', '에서', '여수', '엑스', '포역', '으로', '출발', '위해', '회송', '무궁화호', '열차', '서울', '역진', '입중', '달린', '발전차', '궤도', '이탈', '사고', '따라', '이날', '일반', '열차', '정상운', '행중', '이나', '일부', '경의선', '열차', '운행', '조정', '으로', '지연', '발생', '으로', '알려졌', '코레', '사고', '발생', '즉시', '급복', '구반', '현장', '투입', '복구', '작업', '펼치', '으며', '사고', '원인', '복구', '이후', '조사', '예정', '코레', '열차', '이용', '불편', '드린', '깊이', '사과드린다', '안전', '열차', '운행', '복구', '위해', '최선', '다고', '밝혔']
